In [ ]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import unicodedata
import re
import os
from sqlalchemy import create_engine, text
from dotenv import load_dotenv

df_origin = pd.read_csv("../ventas.csv")
df_test = pd.read_csv("../ventas_limpias.csv")


print(df_test)
# [1.048.574  rows x 11 columns]
# 11 Columnas
# 1.048.575 rows

### Functions
* Create connection
* Cleaning process
* Upload to Postgres DB

In [25]:
# Load .env file
load_dotenv(override=True)

# Envorioment variable
DB_USER = os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASSWORD")
DB_HOST = os.getenv("DB_HOST")
DB_PORT = os.getenv("DB_PORT")
DB_NAME = os.getenv("DB_NAME")

# Create connection
URL = f"postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"
engine = create_engine(URL)

with engine.connect() as conn:
    # Setting schema
    conn.execute(text("SET search_path TO riwi_ventas"))
    
    # Get table list
    print("\nObteniendo lista de tablas...")
    table_list = pd.read_sql(
        text("SELECT table_name FROM information_schema.tables WHERE table_schema = 'riwi_ventas';"), 
        conn
    )
    
    print(f"Encontradas {len(table_list)} tablas:")
    for tabla in table_list["table_name"]:
        print(f"{tabla}")
    
    # Procesing each table
    
    for tabla in table_list["table_name"]:
        print(f"\nProcesando: {tabla}")

        try:
            # Read all the table
            query = text(f'SELECT * FROM riwi_ventas."{tabla}"')
            df = pd.read_sql(query, conn)
            
            print(f"   Filas originales: {len(df):,}")

            # Count nulls
            nulos_antes = df.isnull().sum().sum()  # First .sum() is for columns and the second is for all of them
            if nulos_antes > 0:
                print(f"   Valores nulos encontrados: {nulos_antes:,}")
            
            # Delet rows with null values
            df_limpio = df.dropna()  
            
            # The sum must be cero
            nulos_despues = df_limpio.isnull().sum().sum()
            
            # Calculate rows deleted
            filas_eliminadas = len(df) - len(df_limpio)
            
            print(f"   Filas eliminadas: {filas_eliminadas:,}")
            print(f"   Filas limpias: {len(df_limpio):,}")
            
            if filas_eliminadas > 0:

                print(f"Deleting old data")
                conn.execute(text(f'DELETE FROM riwi_ventas."{tabla}"'))
                conn.commit() 
                
                # Insert cleaned data
                print(f"Saving data...")
                df_limpio.to_sql(
                    name=tabla,
                    con=conn,
                    schema='riwi_ventas',
                    if_exists='append',  # Add to void table
                    index=False
                )
                conn.commit()
                
                print(f"{tabla} Updated succesfull!!")
            else:
                print(f"{tabla} is already cleaned, without changes.")
            
        except Exception as e:
            print(f"Error at table {tabla}: {str(e)}")
            conn.rollback()  # Do a rollback if there is an error
            continue

print("Data Set was cleaned")


Obteniendo lista de tablas...
Encontradas 6 tablas:
tipo_producto
producto
ciudad
factura_ventas
tipo_venta
tipo_cliente

Procesando: tipo_producto
   Filas originales: 7
   Valores nulos encontrados: 1
   Filas eliminadas: 1
   Filas limpias: 6
Deleting old data
Error at table tipo_producto: (psycopg2.errors.ForeignKeyViolation) update o delete en «tipo_producto» viola la llave foránea «producto_tipo_producto_id_fkey» en la tabla «producto»
DETAIL:  La llave (tipo_producto_id)=(1) todavía es referida desde la tabla «producto».

[SQL: DELETE FROM riwi_ventas."tipo_producto"]
(Background on this error at: https://sqlalche.me/e/20/gkpj)

Procesando: producto
   Filas originales: 90
   Valores nulos encontrados: 6
   Filas eliminadas: 6
   Filas limpias: 84
Deleting old data
Error at table producto: (psycopg2.errors.ForeignKeyViolation) update o delete en «producto» viola la llave foránea «factura_ventas_producto_id_fkey» en la tabla «factura_ventas»
DETAIL:  La llave (producto_id)=(1) t

# **REPORTE DE CALIDAD DE DATOS**

## **RESUMEN GENERAL**

| ASPECTO | VALOR | OBSERVACIONES | CALIFICACION |
|---------|-------|---------------|--------------|
| Total Tablas Analizadas | 6 tablas | Todas las tablas del schema riwi_ventas | COMPLETO |
| Total Registros | 1,048,745 registros | Suma de todas las filas de todas las tablas | ALTO VOLUMEN |
| Registros con Problemas | 1,154 registros | Aproximadamente 0.11% del total | BUENA CALIDAD |
| Tablas con Problemas | 4 de 6 tablas | 66% de las tablas tienen algun problema | ATENCION REQUERIDA |
| Principal Problema | Valores Nulos | 1,146 nulos en fechas de ventas | PROBLEMA CRITICO |

---

## **DETALLE POR TABLA**

| NOMBRE TABLA | TOTAL FILAS | FILAS CON PROBLEMAS | % PROBLEMAS | TIPO DE PROBLEMA | RECOMENDACION |
|--------------|-------------|---------------------|-------------|------------------|---------------|
| factura_ventas | 1,048,575 | 1,146 | 0.11% | Fechas nulas (1,146) | ELIMINAR registros |
| ciudad | 34 | 1 | 2.94% | Nombre ciudad nulo | Completar manualmente |
| producto | 90 | 6 | 6.67% | Nombre producto nulo | Completar manualmente |
| tipo_cliente | 5 | 0 | 0.00% | Sin problemas | MANTENER |
| tipo_producto | 7 | 1 | 14.29% | Tipo producto nulo | Completar manualmente |
| tipo_venta | 5 | 1 | 20.00% | Tipo venta nulo | Completar manualmente |

**ESCALA DE CALIDAD:**
- 0-1% = EXCELENTE
- 1-5% = ACEPTABLE  
- 5-10% = REGULAR
- 10% = CRITICO

---

## **TIPOS DE PROBLEMAS ENCONTRADOS**

| TIPO DE PROBLEMA | CANTIDAD | % DEL TOTAL | TABLAS AFECTADAS | GRAVEDAD |
|------------------|----------|-------------|------------------|----------|
| Valores Nulos | 1,155 | 99.9% | factura_ventas, ciudad, producto, tipo_producto, tipo_venta | ALTA |
| Duplicados | 0 | 0% | Ninguna | NULA |
| Formatos Incorrectos | 0 | 0% | Ninguna | NULA |
| Valores Fuera de Rango | 0 | 0% | Ninguna | NULA |
| Relaciones Rotas | 0 | 0% | Ninguna | NULA |